In [ ]:
!rm -rf /content/*
!pip install git+https://github.com/openai/whisper.git 
!sudo apt update && sudo apt install ffmpeg
!pip install --upgrade --no-cache-dir gdown
# https://drive.google.com/file/d/1akD8y6G6pB5YpMcvzPP74yK00kWIa4P3/view?usp=sharing
!gdown https://drive.google.com/uc?id=1akD8y6G6pB5YpMcvzPP74yK00kWIa4P3
!unzip /content/Downloads.zip
!rm -rf /content/Downloads.zip
!git clone https://github.com/x4nth055/gender-recognition-by-voice.git
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install -r /content/gender-recognition-by-voice/requirements.txt
!pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-s4f8iza6
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-s4f8iza6
  Resolved https://github.com/openai/whisper.git to commit 248b6cb124225dd263bb9bd32d060b6517e067f8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=798075 sha256=ff8dc7010c6ea469b1c1102339ee144fd33788b0378d825fc36ec5b7c2191b74
  Stored in directory: /tmp/pip-ephem-wheel-cache-30n5pbbo/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-

In [ ]:
!nvidia-smi

Wed Jun  7 06:45:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**Gender detection**

In [ ]:
%cd /content/gender-recognition-by-voice

/content/gender-recognition-by-voice


In [ ]:
%%writefile train.py
import os
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping

from utils import load_data, split_data, create_model

# load the dataset
X, y = load_data()
# split the data into training, validation and testing sets
data = split_data(X, y, test_size=0.1, valid_size=0.1)
# construct the model
model = create_model()

# use tensorboard to view metrics
tensorboard = TensorBoard(log_dir="logs")
# define early stopping to stop training after 5 epochs of not improving
early_stopping = EarlyStopping(mode="min", patience=5, restore_best_weights=True)

batch_size = 64
epochs = 10

# train the model using the training set and validating using validation set
model.fit(data["X_train"], data["y_train"], epochs=epochs, batch_size=batch_size, validation_data=(data["X_valid"], data["y_valid"]),
          callbacks=[tensorboard, early_stopping])

# save the model to a file
model.save("results/model.h5")

# evaluating the model using the testing set
print(f"Evaluating the model using {len(data['X_test'])} samples...")
loss, accuracy = model.evaluate(data["X_test"], data["y_test"], verbose=0)
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy*100:.2f}%")

Overwriting train.py


In [ ]:
!python train.py

2023-06-07 06:45:26.681966: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 06:45:27.693712: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-07 06:45:30.115716: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-07 06:45:30.585410: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returnin

In [ ]:
%%writefile app.py
import whisper
import torch
import os
import librosa
import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
import streamlit as st
import warnings
warnings.filterwarnings("ignore")

st.sidebar.title("Speech App")
choice = st.sidebar.radio(label = 'Select option',options = ['Audio language detection','Speech to text','English translation','Gender Classification'])

def save_audio(audio_file):
    with open(audio_file.name, "wb") as f:
        f.write(audio_file.getbuffer())

@st.cache(allow_output_mutation=True)
def load_model():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    whisper_model = whisper.load_model("large", device=device)
    model_path = '/content/gender-recognition-by-voice/results/model.h5'
    gender_model = tf.keras.models.load_model(model_path)

    return whisper_model,gender_model

whisper_model,gender_model = load_model()


def extract_feature(file_name, **kwargs):
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    X, sample_rate = librosa.core.load(file_name)
    if chroma or contrast:
        stft = np.abs(librosa.stft(X))
    result = np.array([])
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, chroma))
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
        result = np.hstack((result, mel))
    if contrast:
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, contrast))
    if tonnetz:
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
        result = np.hstack((result, tonnetz))
    return result

def get_gender_prediction(file_path,model):
    features = extract_feature(file_path, mel=True).reshape(1, -1)
    male_prob = model.predict(features)[0][0]
    female_prob = 1 - male_prob
    gender = "male" if male_prob > female_prob else "female"
    if gender == 'male':
        return gender,round(male_prob*100,2)
    else:
        return gender,round(female_prob*100,2)

def language_detection(whisper_model,file_path):
    audio = whisper.load_audio(file_path)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(whisper_model.device)

    # detect the spoken language
    _, probs = whisper_model.detect_language(mel)
    return f"Detected language: {max(probs, key=probs.get)}"

def language_translation(whisper_model,file_path):
    result = whisper_model.transcribe(file_path)
    return result["text"]

def english_translation(whisper_model,file_path):
    result = whisper_model.transcribe(file_path, task = 'translate')
    return result["text"]


if choice == 'Audio language detection':
    st.title('Audio language detection')
    st.text('Detect the speaker language from the audio')
    audio_file = st.file_uploader('Upload audio file here')
    predict = st.button("Predict")
    if predict:
        st.audio(audio_file, format='audio/mp3')
        save_audio(audio_file)
        text = language_detection(whisper_model,audio_file.name)
        st.markdown(text)

elif choice == 'Speech to text':
    st.title('Speech to text')
    st.text('Converts speech to text')
    audio_file = st.file_uploader('Upload audio file here')
    predict = st.button("Predict")
    if predict:
        st.audio(audio_file, format='audio/mp3')
        save_audio(audio_file)
        text = language_translation(whisper_model,audio_file.name)
        st.markdown(f"Output : {text}")

elif choice == 'English translation':
    pass
    st.title('Speech to English text')
    st.text('Converts speech to english text')
    audio_file = st.file_uploader('Upload audio file here')
    predict = st.button("Predict")
    if predict:
        st.audio(audio_file, format='audio/mp3')
        save_audio(audio_file)
        text = english_translation(whisper_model,audio_file.name)
        st.markdown(f"Output : {text}")

else:
    st.title('Audio speaker gender detection')
    st.text('Detects the gender of the speaker in the audio')
    audio_file = st.file_uploader('Upload audio file here')
    predict = st.button("Predict")
    if predict:
        st.audio(audio_file, format='audio/mp3')
        save_audio(audio_file)
        pred,prob_score = get_gender_prediction(audio_file.name,gender_model)
        st.markdown(f"Prediction : {pred}, Prob_score : {prob_score}")

Writing app.py


In [ ]:
!pip install pyngrok
from pyngrok import ngrok
ngrok.set_auth_token('2NEfJKTWX84Cfs0s8vdz1dZkbxt_5ry6aRDP7ZzaZJno1jfYP')

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.


Custom TB Handler failed, unregistering


In [ ]:
!streamlit run app.py --server.port 8111 & npx localtunnel --port 8111


[..................] | fetchMetadata: sill resolveWithNewModule localtunnel@2.0


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8111
  External URL: http://34.143.189.130:8111

npx: installed 22 in 6.82s
your url is: https://tough-melons-appear.loca.lt
2023-06-07 07:54:45.671077: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 07:54:46.658300: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
100%|██████████████████████████████████████| 2.87G/2.87G [00:30<00:00, 102MiB/s]
2023-06-07 07:56:01.054033: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at le